In [11]:
import serial
import time
import binascii
from struct import *
class CySmart:
    Commands = {
        'CMD_Hedder':binascii.unhexlify("4359"),
        'CMD_Footer':binascii.unhexlify("0000"),
        'CMD_INIT_BLE_STACK':binascii.unhexlify("07FC"),
        'CMD_START_SCAN':binascii.unhexlify("93FE"),
        'CMD_STOP_SCAN':binascii.unhexlify("94FE"),
        'CMD_ESTABLISH_CONNECTION':binascii.unhexlify("97FE"),
        'CMD_TERMINATE_CONNECTION':binascii.unhexlify("98FE"),
        'CMD_EXCHANGE_GATT_MTU_SIZE':binascii.unhexlify("12FE"),
        'CMD_READ_USING_CHARACTERISTIC_UUID':binascii.unhexlify("07FE"),
        'CMD_FIND_INCLUDED_SERVICES':binascii.unhexlify("02FE"),
        'CMD_DISCOVER_ALL_CHARACTERISTICS':binascii.unhexlify("03FE")
    }
    
    
    CYSMT_EVT_HEADER_CODE = binascii.unhexlify("BDA7")
    EVT_SCAN_PROGRESS_RESULT = binascii.unhexlify("8A06")
    EVT_COMMAND_STATUS =  binascii.unhexlify("7E04")
    EVT_COMMAND_COMPLETE =  binascii.unhexlify("7F04")

    def __init__(self, ComPort='\\.\COM6'):
        self.serin = serial.Serial(ComPort, 115200, timeout=3)
        self.serin.isOpen()

    def hexPrint(self,s):
        return ":".join("{:02x}".format(ord(c)) for c in s)
    
    def hexArray(self,s):
        return self.hexPrint(s).split(":")
    
    def sendCommand(self,command,timeout = 3):
        self.serin.write(self.Commands['CMD_Hedder']+command+self.Commands['CMD_Footer'])
        self.serin.flush()
        time.sleep(.1)
        return self.prossesOutput()
    
    
    def prossesOutput(self):
        payloads = {}
        while self.serin.inWaiting():
            hedderTest = self.serin.read(2)
            if self.CYSMT_EVT_HEADER_CODE in hedderTest:
                #Have Hedder message
                mesageLen = self.serin.read(2)
                mesageLen = unpack('h',mesageLen)[0]
                message = self.serin.read(mesageLen)
                if len(message) > 4:
                # have message                   
                    event, command = unpack('2s2s', message[0:4])
                    body = message[4:]
                    if event not in payloads:
                         payloads[event] = []      
                    payloads[event].append({'cmd':command, 'body':body})
            else:
                time.sleep(.1)
        return payloads
        
    def getScanData(self,cyd):
        Ble = {'address':[], 'name':""}
        
        if self.EVT_SCAN_PROGRESS_RESULT in cyd:
            for scan in cyd[self.EVT_SCAN_PROGRESS_RESULT]:
                if len(scan['body']) > 10:
                    inputString = scan['body']
                    Ble['address'] =  self.hexArray(inputString)[1:7]
                    if '\t' in inputString:
                        nm_length=  int(self.hexArray(inputString.split('\t')[0])[-1],16)-1
                        Ble['name'] = inputString.split('\t')[1][0:nm_length]
        return Ble
    def openConection(self,address):
        
        ads = "".join("{:s}".format(c) for c in address)
        # Address type is followed by address
        ads = self.Commands['CMD_ESTABLISH_CONNECTION'] +"0700"+ ads
        print self.sendCommand(ads)
        
        #CYBLE_GATT_XCHG_MTU_PARAM #4359 12FE 04 0004000002
        #CMD_READ_USING_CHARACTERISTIC_UUID 4359 07FE 09 0004 0001 002A0100FFFF
    def close_Conection(self):
        #CMD_TERMINATE_CONNECTION 4359 98FE 02000400
        pass
    
    def close(self):
        self.serin.close()
        
        
cy = CySmart()
cy.sendCommand(cy.Commands['CMD_INIT_BLE_STACK'])
cyd= cy.sendCommand(cy.Commands['CMD_START_SCAN'])
time.sleep(3)
cy.sendCommand(cy.Commands['CMD_STOP_SCAN'])
cy.close()

print cy.getScanData(cyd)

{'name': 'BLE GamePad', 'address': ['1d', '2a', '05', '50', 'a0', '00']}


In [8]:

        print cy.getScanData(scan['body'])

{'name': 'BLE GamePad', 'address': ['1d', '2a', '05', '50', 'a0']}


In [14]:
test = ['1d', '2a', '05', '50', 'a0']
print

1d2a0550a0


In [7]:

resdt2 = b'\x7E\x04\x07\xFC\x00\x00'
unpack('2s2sxx',resdt2)


('~\x04', '\x07\xfc')